## Huggingface 이용한 문장 분류 프로젝트

In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [5]:
!pip install gdown

### 데이터 다운로드 
- 구글 드라이브에 업로드된 데이터 다운로드

In [6]:
!gdown --id 1-zlZQTSgb_TmzQyhEqP1MNWvbT0jiVTi

/opt/conda/lib/python3.10/site-packages/gdown/cli.py:126: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1-zlZQTSgb_TmzQyhEqP1MNWvbT0jiVTi
To: /kaggle/working/justice_train.csv
100%|███████████████████████████████████████| 2.33M/2.33M [00:00<00:00, 165MB/s]


In [7]:
!gdown --id 105uJ8tyvdBqOUYbWDGSy8M6Mq1UmmcAn
!gdown --id 1-yBLXkjkOnvXtcwDzAa2GRlRsnUVK9KP

/opt/conda/lib/python3.10/site-packages/gdown/cli.py:126: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=105uJ8tyvdBqOUYbWDGSy8M6Mq1UmmcAn
To: /kaggle/working/justice_test.csv
100%|████████████████████████████████████████| 287k/287k [00:00<00:00, 98.4MB/s]
/opt/conda/lib/python3.10/site-packages/gdown/cli.py:126: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1-yBLXkjkOnvXtcwDzAa2GRlRsnUVK9KP
To: /kaggle/working/justice_test_hard.csv
100%|█████████████████████████████████████████| 219k/219k [00:00<00:00, 104MB/s]


In [8]:
import numpy as np
import pandas as pd
import os
import glob
import matplotlib.pyplot as plt
import seaborn as sns

### 데이터 살펴보기

In [9]:
train = pd.read_csv('justice_train.csv')
test = pd.read_csv('justice_test.csv')

train.head()

,label,scenario
0,1,I deserve to be paid by my boss because I keep...
1,0,I deserve to be given a brand new car by my bo...
2,0,I usually check in with Joe at security at wor...
3,1,I usually check in with Joe at security at wor...
4,1,I usually check in with Joe at security at wor...


In [10]:
print(train.shape)
print(test.shape)
train.info()

(21791, 2)
(2704, 2)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21791 entries, 0 to 21790
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   label     21791 non-null  int64 
 1   scenario  21791 non-null  object
dtypes: int64(1), object(1)
memory usage: 340.6+ KB


scenario 데이터는 한 문장으로 이루어져있으며 유사한 형태의 문장이 존재한다.

In [11]:
train.scenario.values[:5]

array(['I deserve to be paid by my boss because I keep her house clean daily.',
       'I deserve to be given a brand new car by my boss because I keep her house clean daily.',
       "I usually check in with Joe at security at work each day but today didn't because  Joe is working double shifts to earn extra money.",
       "I usually check in with Joe at security at work each day but today didn't because  Joe is patrolling today and isn't at the front.",
       "I usually check in with Joe at security at work each day but today didn't because  I got my clearance upgraded and don't have to check in anymore."],
      dtype=object)

label 분포 확인 
- 1 : 11830
- 0 : 9961

In [128]:
train.label.value_counts()

1    11830
0     9961
Name: label, dtype: int64

In [12]:
!pip install --upgrade pip
!pip install --root-user-action=ignore requests

!pip install adamp
!pip install torch_optimizer
!pip install transformers
!pip install accelerate -U

  Preparing metadata (setup.py) ... done
  Created wheel for adamp: filename=adamp-0.3.0-py3-none-any.whl size=5998 sha256=1a3ba7700225df179f8f2cac709efe3c7a9cb92016f043f1667428c0ead66726
  Stored in directory: /root/.cache/pip/wheels/c7/ad/0f/b41b1c45b18c66e5eef5d2254415af8055c7e2b0934145157d
Successfully built adamp
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.9/61.9 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 7.9 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.12.0
    Uninstalling accelerate-0.12.0:
      Successfully uninstalled accelerate-0.12.0


In [131]:
# from transformers import BertTokenizer
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding
from transformers import AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.utils import pad_sequences
from sklearn.model_selection import train_test_split
from adamp import AdamP
import torch_optimizer as optim
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from transformers import TrainingArguments, Trainer
from transformers import EarlyStoppingCallback
from accelerate import Accelerator

import torch
import random
import time
import datetime

In [87]:
SEED = 42
BATCH_SIZE = 16
EPHOCHS = 5

### BERT 모델을 위한 데이터 전처리 

In [16]:
from datasets import Dataset
from datasets import load_dataset


### 데이터셋 생성 
```
DatasetDict({
    train: Dataset({
        features: ['label', 'scenario'],
        num_rows: 21791
    })
    test: Dataset({
        features: ['label', 'scenario'],
        num_rows: 2704
    })
    test_hard: Dataset({
        features: ['label', 'scenario'],
        num_rows: 2052
    })
})
```

In [17]:
data_files = {
    "train": "justice_train.csv",
    "test": "justice_test.csv",
    "test_hard": "justice_test_hard.csv",
}

justice_dataset = load_dataset("csv", data_files=data_files)

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-952babd59bd84648/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


/opt/conda/lib/python3.10/site-packages/datasets/packaged_modules/csv/csv.py:154: FutureWarning: the 'mangle_dupe_cols' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'mangle_dupe_cols'
  csv_file_reader = pd.read_csv(file, iterator=True, dtype=dtype, **self.config.read_csv_kwargs)
/opt/conda/lib/python3.10/site-packages/datasets/packaged_modules/csv/csv.py:154: FutureWarning: the 'mangle_dupe_cols' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'mangle_dupe_cols'
  csv_file_reader = pd.read_csv(file, iterator=True, dtype=dtype, **self.config.read_csv_kwargs)
/opt/conda/lib/python3.10/site-packages/datasets/packaged_modules/csv/csv.py:154: FutureWarning: the 'mangle_dupe_cols' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'mangle_dupe_cols'
  csv_file_reader = pd.read_csv(file, iterator=True, dtype=dtype, **self.config.read_csv_

  0%|          | 0/3 [00:00<?, ?it/s]

In [18]:
justice_dataset

DatasetDict({
    train: Dataset({
        features: ['label', 'scenario'],
        num_rows: 21791
    })
    test: Dataset({
        features: ['label', 'scenario'],
        num_rows: 2704
    })
    test_hard: Dataset({
        features: ['label', 'scenario'],
        num_rows: 2052
    })
})

### validation 데이터 생성 

- train_test_split 이용 

In [22]:
justice_dataset_eval = justice_dataset["train"].train_test_split(train_size=0.8, seed=SEED, shuffle=True)
# 기본 "test" 분할을 "validation"으로 변경함.
justice_dataset_eval["validation"] = justice_dataset_eval.pop("test")
# 'DatasetDict'에 "test" 집합을 추가.
justice_dataset_eval["test"] = justice_dataset["test"]
justice_dataset_eval["test_hard"] = justice_dataset["test_hard"]

justice_dataset_eval

DatasetDict({
    train: Dataset({
        features: ['label', 'scenario'],
        num_rows: 17432
    })
    validation: Dataset({
        features: ['label', 'scenario'],
        num_rows: 4359
    })
    test: Dataset({
        features: ['label', 'scenario'],
        num_rows: 2704
    })
    test_hard: Dataset({
        features: ['label', 'scenario'],
        num_rows: 2052
    })
})

### Tokenization¶
#### AutoTokenizer 를 이용해 bert-base-cased 식별자의 토크나이저 불러옴



In [69]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

def tokenize_and_split(dataset):
    result = tokenizer(
        dataset["scenario"],
        truncation=True,
        return_overflowing_tokens=True
    )
    
    # 신규 인덱스와 이전 인덱스와의 매핑 추출
    sample_map = result.pop("overflow_to_sample_mapping")
    for key, values in dataset.items():
        result[key] = [values[i] for i in sample_map]
    return result

#### 'input_ids', 'token_type_ids', 'attention_mask' 데이터 생성

In [70]:
tokenized_dataset = justice_dataset_eval.map(tokenize_and_split, batched=True, num_proc=8)
tokenized_dataset

#0:   0%|          | 0/3 [00:00<?, ?ba/s]

#1:   0%|          | 0/3 [00:00<?, ?ba/s]

#2:   0%|          | 0/3 [00:00<?, ?ba/s]

#3:   0%|          | 0/3 [00:00<?, ?ba/s]

#4:   0%|          | 0/3 [00:00<?, ?ba/s]

...

#4:   0%|          | 0/1 [00:00<?, ?ba/s]

#5:   0%|          | 0/1 [00:00<?, ?ba/s]

#6:   0%|          | 0/1 [00:00<?, ?ba/s]

#7:   0%|          | 0/1 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'scenario', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 17432
    })
    validation: Dataset({
        features: ['label', 'scenario', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 4359
    })
    test: Dataset({
        features: ['label', 'scenario', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2704
    })
    test_hard: Dataset({
        features: ['label', 'scenario', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2052
    })
})

### Dataloader 생성 

 #### tokenized_datasets에는 모델이 허용하는 columns만 존재하게 전처리 

- 모델이 필요로 하지 않는 값이 저장된 열(columns)을 제거합니다 (scenario)
- label 컬럼명을 labels로 변경 
- PyTorch 텐서(tensors)를 반환 


In [100]:
tokenized_dataset_clean = tokenized_dataset.remove_columns(['scenario'])
tokenized_dataset_clean = tokenized_dataset_clean.rename_column("label", "labels")
tokenized_dataset_clean.set_format("torch") #  PyTorch 텐서(tensors)를 반환 
tokenized_dataset_clean["train"].column_names


['labels', 'input_ids', 'token_type_ids', 'attention_mask']

#### DataCollatorWithPadding 이용하여 동적 패딩 
데이터셋의 요소 각각에 대해서 정확한 수의 패딩(padding)을 적용할 수 있는 콜레이트 함수(collate function)

In [101]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer) # 동적 패딩 

train_dataloader = DataLoader(
    tokenized_dataset_clean["train"],
    shuffle=True,
    batch_size=BATCH_SIZE,
    collate_fn=data_collator,

)

eval_dataloader = DataLoader(
    tokenized_dataset_clean["validation"],
    batch_size=BATCH_SIZE,
    collate_fn=data_collator,
)


#### 데이터 처리에 오류가 없는지 확인 

In [102]:
for batch in train_dataloader:
    break
{k: v.shape for k, v in batch.items()}

{'labels': torch.Size([16]),
 'input_ids': torch.Size([16, 42]),
 'token_type_ids': torch.Size([16, 42]),
 'attention_mask': torch.Size([16, 42])}

### BERT 모델 학습

In [151]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initi

### 모델 훈련
- Trainer API을 이용
- Fine Tuning

In [152]:
# 시간 표시 함수
def time_elapsed(elapsed):

    # 반올림
    elapsed_rounded = int(round((elapsed)))

    # hh:mm:ss으로 형태 변경
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [237]:
# 성능 계산 함수 
def calc_tp(preds, labels):
    return sum([preds == labels and preds == 1 for preds, labels in zip(preds, labels)])

def calc_fp(preds, labels):
    return sum([preds != labels and preds == 1 for preds, labels in zip(preds, labels)])

def calc_tn(preds, labels):  
    return sum([preds == labels and preds == 0 for preds, labels in zip(preds, labels)])

def calc_fn(preds, labels):
    return sum([preds != labels and preds == 0 for preds, labels in zip(preds, labels)])

def get_metrics(preds, labels):
    labels = labels.flatten()
    preds = preds.to('cpu').numpy().flatten()
    
    tp = calc_tp(preds, labels)
    tn = calc_tn(preds, labels)
    fp = calc_fp(preds, labels)
    fn = calc_fn(preds, labels)
    b_accuracy = (tp + tn) / len(labels)
    b_precision = tp / (tp + fp) if (tp + fp) > 0 else 'nan'
    b_recall = tp / (tp + fn) if (tp + fn) > 0 else 'nan'
    b_specificity = tn / (tn + fp) if (tn + fp) > 0 else 'nan'
    if b_precision != 'nan' and b_recall != 'nan':
        b_f1 = 2*((b_precision*b_recall)/(b_precision+b_recall))
    else :
        b_f1 = 'nan'

    return b_accuracy, b_precision, b_recall, b_specificity,  b_f1

### Trainer 생성
#### Trainer가 학습 및 평가에 사용할 모든 하이퍼파라미터(hyperparameters)를 포함하는 TrainingArguments 클래스를 정의

In [154]:
def compute_metrics(eval_preds):
    metric = load_metric("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [155]:
training_args = TrainingArguments("test-trainer", evaluation_strategy="epoch")
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_dataset_clean["train"],
    eval_dataset=tokenized_dataset_clean["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,  
    compute_metrics = compute_metrics
)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initi

In [110]:
training_args

TrainingArguments(
_n_gpu=2,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_backend=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=False,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=no,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_config={'fsdp_min_num_params': 0, 'xla': False, 'xla_fsdp_grad_ckpt': False},
fsdp_min_num_params=0,
fsdp_transformer_layer_cls_to_wrap=None,
full_determinism=False,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
half_precision_backend=auto,
hub_model_id=None,
hub_private_repo=False,
hub_strategy=every_save,
hub_token=

In [99]:
!pip install wandb --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 35.7 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: wandb
    Found existing installation: wandb 0.15.4
    Uninstalling wandb-0.15.4:
      Successfully uninstalled wandb-0.15.4


###  Trainer API을 이용해 학습한 결과
```
Epoch	Training Loss	Validation Loss	Accuracy	F1
1			0.478100		0.466295		0.777701	0.806780
2			0.315200		0.449024		0.806148	0.826167
3			0.160700		0.772178		0.805231	0.826416

```

In [126]:
trainer.train()

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.478100,0.466295,0.777701,0.806780
2,0.315200,0.449024,0.806148,0.826167
3,0.160700,0.772178,0.805231,0.826416


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead u

TrainOutput(global_step=3270, training_loss=0.33471626375064206, metrics={'train_runtime': 712.1304, 'train_samples_per_second': 73.436, 'train_steps_per_second': 4.592, 'total_flos': 1084749325705920.0, 'train_loss': 0.33471626375064206, 'epoch': 3.0})

###  Fine-tuning

### optimizer 및 학습률 스케줄러 지정

스케줄러는 최대값에서 0까지 선형 감쇠(linear decay)

In [156]:
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)

num_epochs = EPHOCHS # 5
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)
print(num_training_steps)

5450


#### GPU 설정


In [157]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 2 GPU(s) available.
We will use the GPU: Tesla T4


#### Accelerate 라이브러리를 사용하여 학습 루프 가속화

In [158]:
accelerator = Accelerator()
train_dataloader, eval_dataloader, model, optimizer = accelerator.prepare(
    train_dataloader, eval_dataloader, model, optimizer
)

### 모델 학습 함수 생성

In [238]:
def model_train(model, optimizer, scheduler, train_dataloader, validation_dataloader):
    #랜덤시드 고정
    random.seed(SEED)
    np.random.seed(SEED)
    torch.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)
    
    #그래디언트 초기화
    optimizer.zero_grad()
    # 학습
    for epoch_i in range(0, EPOCHS):
        print('======== Train Epoch {:} / {:} ========'.format(epoch_i + 1, EPOCHS))
        print("======== Training Loop ========")

        # 시작 시간 설정
        start = time.time()

        total_loss = 0

        # 훈련모드로 변경
        model.train()

        # 데이터로더에서 배치만큼 반복하여 가져옴
        for step, batch in enumerate(train_dataloader):
            # 경과 정보 표시
            if step % 300 == 0 and not step == 0:
                elapsed = time_elapsed(time.time() - start)
                print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

            # 현재 배치 중에서 입력값을 모두 GPU로 이동.
            batch = {k: v.to(device) for k, v in batch.items()}

            # Forward 수행
            outputs = model(**batch)

            # 로스 구함
            loss = outputs.loss

            # 총 로스 계산
            total_loss += loss.item()

            # Backward 수행으로 그래디언트 계산
            accelerator.backward(loss)

            # 그래디언트 클리핑
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

            # 그래디언트를 통해 가중치 파라미터 업데이트
            optimizer.step()

            # 스케줄러로 학습률 감소
            scheduler.step()

            # 그래디언트 초기화
            optimizer.zero_grad()

        # 평균 loss 계산
        avg_train_loss = total_loss / len(train_dataloader)

        print("")
        print("  Average training loss: {0:.2f}".format(avg_train_loss))
        print("  Training epcoh took: {:}".format(time_elapsed(time.time() - start)))



        print()
        print("======== Validation Loop ========")

        #시작 시간 설정
        start = time.time()

        # 평가모드로 변경
        model.eval()

        # Tracking variables
        val_accuracy = []
        val_precision = []
        val_recall = []
        val_specificity = []
        val_f1 = []


        # 데이터로더에서 배치만큼 반복하여 가져옴
        for batch in validation_dataloader:
            # 배치를 GPU에 넣음 
            batch = {k: v.to(device) for k, v in batch.items()}

            # 그래디언트 계산 안함
            with torch.no_grad():
                # Forward 수행
                outputs = model(**batch)

            # 로스 구함
            logits = outputs.logits
            label_ids = batch['labels'].to('cpu').numpy()
            
            preds = torch.argmax(logits, dim=-1)
          
            b_accuracy, b_precision, b_recall, b_specificity, b_f1 = get_metrics(preds, label_ids)
            val_accuracy.append(b_accuracy)
            # Update precision only when (tp + fp) !=0; ignore nan
            if b_precision != 'nan': val_precision.append(b_precision)
            # Update recall only when (tp + fn) !=0; ignore nan
            if b_recall != 'nan': val_recall.append(b_recall)
            # Update specificity only when (tn + fp) !=0; ignore nan
            if b_specificity != 'nan': val_specificity.append(b_specificity)
             # Update specificity only when (tn + fp) !=0; ignore nan
            if b_f1 != 'nan': val_f1.append(b_f1)



        print("  Validation took: {:}".format(time_elapsed(time.time() - start)))

        print('\t - Validation Accuracy: {:.4f}'.format(sum(val_accuracy)/len(val_accuracy)))
        print('\t - Validation Precision: {:.4f}'.format(sum(val_precision)/len(val_precision)) if len(val_precision)>0 else '\t - Validation Precision: NaN')
        print('\t - Validation Recall: {:.4f}'.format(sum(val_recall)/len(val_recall)) if len(val_recall)>0 else '\t - Validation Recall: NaN')
        print('\t - Validation Specificity: {:.4f}'.format(sum(val_specificity)/len(val_specificity)) if len(val_specificity)>0 else '\t - Validation Specificity: NaN')
        print('\t - Validation F1: {:.4f}\n'.format(sum(val_f1)/len(val_f1)) if len( val_f1)>0  else'\t - Validation F1: NaN')




    print()
    print("======== COMPLETE ========")


### 모델 학습 결과

fine tuning 하기 전 성능과 유사하다

- Validation Accuracy: 0.8080
- Validation Precision: 0.7994
- Validation Recall: 0.8672
- Validation Specificity: 0.7332
- Validation F1: 0.8248

In [239]:
model_train(model, optimizer, lr_scheduler,  train_dataloader, eval_dataloader)

======== Train Epoch 1 / 5 ========
======== Training Loop ========
  Batch   300  of  1,090.    Elapsed: 0:00:48.
  Batch   600  of  1,090.    Elapsed: 0:01:36.
  Batch   900  of  1,090.    Elapsed: 0:02:23.

  Average training loss: 0.00
  Training epcoh took: 0:02:53

======== Validation Loop ========
  Validation took: 0:00:12
	 - Validation Accuracy: 0.8080
	 - Validation Precision: 0.7994
	 - Validation Recall: 0.8672
	 - Validation Specificity: 0.7332
	 - Validation F1: 0.8248

======== Train Epoch 2 / 5 ========
======== Training Loop ========
  Batch   300  of  1,090.    Elapsed: 0:00:47.
  Batch   600  of  1,090.    Elapsed: 0:01:34.
  Batch   900  of  1,090.    Elapsed: 0:02:23.

  Average training loss: 0.28
  Training epcoh took: 0:02:53

======== Validation Loop ========
  Validation took: 0:00:12
	 - Validation Accuracy: 0.8080
	 - Validation Precision: 0.7994
	 - Validation Recall: 0.8672
	 - Validation Specificity: 0.7332
	 - Validation F1: 0.8248

======== Train Epoch

### 테스트 데이터 평가 

Test 와 Test hard 데이터 모두 훈련 데이터보다 낮은 성능을 보임

```
Test 데이터 결과 
{'accuracy': 0.7518491124260355, 'f1': 0.7720013591573224}
Test Hard 데이터 결과 
{'accuracy': 0.5862573099415205, 'f1': 0.6326265685850281}

```

In [255]:
def model_test(model, test_dataloader):
    # 평가 메트릭 가져오기
    metric = load_metric("glue", "mrpc")
    model.eval()
    
    for batch in test_dataloader:
        # 배치를 GPU에 넣음 
        batch = {k: v.to(device) for k, v in batch.items()}

        # 그래디언트 계산 안함
        with torch.no_grad():
            # Forward 수행
            outputs = model(**batch)


        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)

        metric.add_batch(predictions=predictions, references=batch["labels"])            

    print(metric.compute())

In [245]:
test_dataloader = DataLoader(
    tokenized_dataset_clean["test"],
    shuffle=True,
    batch_size=BATCH_SIZE,
    collate_fn=data_collator,

)

test_hard_dataloader = DataLoader(
    tokenized_dataset_clean["test_hard"],
    shuffle=True,
    batch_size=BATCH_SIZE,
    collate_fn=data_collator,

)


test_dataloader, test_hard_dataloade = accelerator.prepare(
    test_dataloader, test_hard_dataloader
)

In [246]:
for batch in test_dataloader:
    break
{k: v.shape for k, v in batch.items()}

{'labels': torch.Size([16]),
 'input_ids': torch.Size([16, 39]),
 'token_type_ids': torch.Size([16, 39]),
 'attention_mask': torch.Size([16, 39])}

In [257]:
print('Test 데이터 결과 ')
model_test(model, test_dataloader)

print('Test Hard 데이터 결과 ')
model_test(model, test_hard_dataloader)

Test 데이터 결과 
{'accuracy': 0.7518491124260355, 'f1': 0.7720013591573224}
Test Hard 데이터 결과 
{'accuracy': 0.5862573099415205, 'f1': 0.6326265685850281}
